<a href="https://colab.research.google.com/github/Vincenzo-Miracula/MasterSDS/blob/main/bluesky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install atproto

In [46]:
from atproto import Client
import requests
import json
from tqdm import tqdm
import time
import pandas as pd
import configparser
from datetime import datetime

In [4]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [ ]:
client = Client()
# By default, it uses the server of bsky.app. To change this behavior, pass the base api URL to constructor
client_id = config['bluesky']['client_id']
password_app = config['bluesky']['password_app']

client.login(client_id, password_app)

In [6]:
token = ''

In [ ]:
# URL dell'API
url = "https://milkcap.us-west.host.bsky.network/xrpc/app.bsky.feed.searchPosts"
headers = {"Authorization": f"Bearer {token}"}

# Parametri della richiesta
params = {
    "q": "",  # Query di ricerca
    "limit": 100,    # Limite massimo per ogni richiesta
    "sort": "latest", # Ordinamento dei risultati - top o latest
    "tag": '#',  # Filtro per il tag
}

def estrai_post(url, params, max_posts=400):
    all_posts = []
    cursor = None

    with tqdm(total=max_posts, desc="Estrazione post") as pbar:
        while len(all_posts) < max_posts:
            if cursor:
                params["cursor"] = cursor  # Usa il cursore per la paginazione
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                all_posts.extend(data["posts"])
                pbar.update(len(data["posts"]))  # Aggiorna la barra di avanzamento

                cursor = data.get("cursor", None)
                if cursor is None:
                    break
            else:
                print(f"Errore nella richiesta: {response.status_code}")
                break

            time.sleep(1)  # Pausa prima della prossima richiesta

    return all_posts

data = estrai_post(url, params)
print(f"Numero di post estratti: {len(data)}")

In [ ]:
data[0]

In [ ]:
for post in data:
  print(post['record']['createdAt'])

In [38]:
def estrai_displaynames_e_testi(data):
    risultati = []
    for post in data:

      handle = post['author']['handle']
      displayname = post['author'].get('displayName', 'empty')
      text = post['record']['text']
      date = post['record']['createdAt']
      risultati.append((handle, displayname, text, date))
    return risultati

In [ ]:
for item in data:
    print(item.keys())[0]

In [42]:
content = estrai_displaynames_e_testi(data)

In [ ]:
len(pd.DataFrame(content))

400

In [58]:
df = pd.DataFrame(content, columns=['Author', 'Username', 'Text', 'Date'])

In [ ]:
df['Date'] = df['Date'].str.split('T').str[0]
df['Date']

In [62]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.info()